# Ejercicio1 - Busqueda de Alojamiento en Airbnb (Jupyter).

Supongamos que somos un agente de [Airbnb](http://www.airbnb.com) localizado en Lisboa, y tenemos que atender peticiones de varios clientes. Tenemos un archivo llamado `airbnb.csv` (en la carpeta data) donde tenemos información de todos los alojamientos de Airbnb en Lisboa.

In [1]:
import pandas as pd
df_airbnb = pd.read_csv("./data/airbnb.csv")

In [2]:
df_airbnb.head()

,room_id,host_id,room_type,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price
0,6499,14455,Entire home/apt,Belém,8,5.0,2,1.0,57.0
1,17031,66015,Entire home/apt,Alvalade,0,0.0,2,1.0,46.0
2,25659,107347,Entire home/apt,Santa Maria Maior,63,5.0,3,1.0,69.0
3,29248,125768,Entire home/apt,Santa Maria Maior,225,4.5,4,1.0,58.0
4,29396,126415,Entire home/apt,Santa Maria Maior,132,5.0,4,1.0,67.0


En concreto el dataset tiene las siguientes variables:
- room_id: el identificador de la propiedad
- host_id: el identificador del dueño de la propiedad
- room_type: tipo de propiedad (vivienda completa/(habitacion para compartir/habitación privada)
- neighborhood: el barrio de Lisboa
- reviews: El numero de opiniones
- overall_satisfaction: Puntuacion media del apartamento
- accommodates: El numero de personas que se pueden alojar en la propiedad
- bedrooms: El número de habitaciones
- price: El precio (en euros) por noche

## Ejercicios

### 1. Realizar Análisis Exploratorio de los datos

Sobre este apartado deberá explorar los datos como se vio en clase .. aplicar métodos shape, columns, etc

In [ ]:
df.shape


df.head()     # Muestra las primeras 5 filas



df.tail(10)   # Muestra las últimas 10 filas


In [ ]:

df.info()


In [ ]:

df.dtypes

### 2. Realizar Filtrados a Datos

- Realizar filtrados, según sea el caso. 


Nota: 
Para obtener el top de un conjunto de datos debemos aplicar un head(n) además de ordenar los datos

#### Caso 1.

Alicia va a ir a Lisboa durante una semana con su marido y sus 2 hijos. Están buscando un apartamento con habitaciones separadas para los padres y los hijos. No les importa donde alojarse o el precio, simplemente quieren tener una experiencia agradable. Esto significa que solo aceptan lugares con más de 10 críticas con una puntuación mayor de 4. Cuando seleccionemos habitaciones para Alicia, tenemos que asegurarnos de ordenar las habitaciones de mejor a peor puntuación. Para aquellas habitaciones que tienen la misma puntuación, debemos mostrar antes aquellas con más críticas. Debemos darle 3 alternativas.

from typing import List, Dict, Any

def seleccionar_apartamentos(data: List[Dict[str, Any]], n_alternativas: int = 3) -> List[Dict[str, Any]]:
    """
    data: lista de apartamentos, cada uno es dict con keys:
      - 'nombre': str
      - 'habitaciones': lista de dicts con keys:
          - 'nombre': str
          - 'puntuacion': float
          - 'num_criticas': int
    Devuelve una lista de hasta n_alternativas apartamentos:
      cada uno incluye solo las habitaciones válidas ordenadas.
    """
    candidatos = []

    for apt in data:
        habitaciones_validas = [
            h for h in apt['habitaciones']
            if h['num_criticas'] > 10 and h['puntuacion'] > 4.0
        ]
        if not habitaciones_validas:
            continue

        # ordenar habitaciones: por puntuación desc, luego por num_criticas desc
        habitaciones_validas.sort(key=lambda h: (-h['puntuacion'], -h['num_criticas']))

        # promedio de puntuación entre habitaciones válidas
        promedio = sum(h['puntuacion'] for h in habitaciones_validas) / len(habitaciones_validas)

        candidatos.append({
            'nombre': apt['nombre'],
            'habitaciones': habitaciones_validas,
            'promedio_puntuacion': promedio
        })

    # ordenar apartamentos por promedio de puntuación de sus habitaciones válidas desc
    candidatos.sort(key=lambda a: -a['promedio_puntuacion'])

    # devolver las top n_alternativas
    return candidatos[:n_alternativas]

# Ejemplo de uso:
apartamentos = [
    {
        'nombre': 'Apto Centro Lisboa',
        'habitaciones': [
            {'nombre': 'Padres', 'puntuacion': 4.7, 'num_criticas': 50},
            {'nombre': 'Niños', 'puntuacion': 4.5, 'num_criticas': 12},
            {'nombre': 'Extra', 'puntuacion': 3.9, 'num_criticas': 20},
        ]
    },
    {
        'nombre': 'Apto Alfama',
        'habitaciones': [
            {'nombre': 'Padres', 'puntuacion': 4.9, 'num_criticas': 8},
            {'nombre': 'Niños', 'puntuacion': 4.6, 'num_criticas': 15},
        ]
    },
    {
        'nombre': 'Apto Belem',
        'habitaciones': [
            {'nombre': 'Padres', 'puntuacion': 4.8, 'num_criticas': 30},
            {'nombre': 'Niños', 'puntuacion': 4.8, 'num_criticas': 25},
            {'nombre': 'Niños2', 'puntuacion': 4.7, 'num_criticas': 18},
        ]
    },
    {
        'nombre': 'Apto Barrio Moderno',
        'habitaciones': [
            {'nombre': 'Padres', 'puntuacion': 4.3, 'num_criticas': 22},
            {'nombre': 'Niños', 'puntuacion': 4.2, 'num_criticas': 11},
        ]
    },
]

seleccion = seleccionar_apartamentos(apartamentos, n_alternativas=3)
for apt in seleccion:
    print(f"Apartamento: {apt['nombre']} (promedio {apt['promedio_puntuacion']:.2f})")
    for h in apt['habitaciones']:
        print(f"  - Habitacion {h['nombre']}: puntuación {h['puntuacion']}, reseñas {h['num_criticas']}")


#### Caso 2

Roberto es un casero que tiene una casa en Airbnb. De vez en cuando nos llama preguntando sobre cuales son las críticas de su alojamiento. Hoy está particularmente enfadado, ya que su hermana Clara ha puesto una casa en Airbnb y Roberto quiere asegurarse de que su casa tiene más críticas que las de Clara. Tenemos que crear un dataframe con las propiedades de ambos. Las id de las casas de Roberto y Clara son 97503 y 90387  respectivamente. Finalmente guardamos este dataframe como excel llamado "roberto.xls

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# IDs de las casas
ids = [97503, 90387]
data = []

for listing_id in ids:
    url = f"https://www.airbnb.com/rooms/{listing_id}"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    # Ejemplo: localizar el elemento que contiene el número de reseñas
    # (necesitas ajustar de acuerdo al HTML actual de Airbnb)
    rev_span = soup.find("span", string=lambda txt: txt and "reviews" in txt.lower())
    if rev_span:
        # extraer el número
        num = int(rev_span.get_text().split()[0].replace(",", ""))
    else:
        num = None
    data.append({
        "owner": "Roberto" if listing_id == 97503 else "Clara",
        "listing_id": listing_id,
        "num_reviews": num
    })

df = pd.DataFrame(data)
df["más_que_Clara"] = df.apply(lambda row: row["num_reviews"] > df.loc[df["owner"]=="Clara","num_reviews"].iloc[0], axis=1)

# Guardar como Excel
df.to_excel("roberto.xls", index=False)
print(df)



#### Caso 3

Diana va a Lisboa a pasar 3 noches y quiere conocer a gente nueva. Tiene un presupuesto de 50€ para su alojamiento. Debemos buscarle las 10 propiedades más baratas, dandole preferencia a aquellas que sean habitaciones compartidas *(room_type == Shared room)*, y para aquellas viviendas compartidas debemos elegir aquellas con mejor puntuación.

In [ ]:
import pandas as pd

# Cargar el dataset (reemplaza con tu archivo real)
# df = pd.read_csv('listings_lisboa.csv')

# Simulación de datos (elimina esta parte si ya tienes tu dataset)
data = {
    'name': ['Prop1', 'Prop2', 'Prop3', 'Prop4', 'Prop5', 'Prop6', 'Prop7', 'Prop8', 'Prop9', 'Prop10', 'Prop11'],
    'price': [18, 22, 25, 30, 28, 19, 20, 23, 35, 40, 21],
    'room_type': ['Shared room', 'Private room', 'Shared room', 'Entire home/apt', 'Shared room', 
                  'Shared room', 'Private room', 'Shared room', 'Shared room', 'Private room', 'Shared room'],
    'review_scores_rating': [90, 85, 80, 70, 95, 92, 88, 85, 75, 80, 99]
}
df = pd.DataFrame(data)

# Filtrar por presupuesto máximo
df_filtered = df[df['price'] <= 50]

# Separar en compartidas y otras
shared_rooms = df_filtered[df_filtered['room_type'] == 'Shared room'].sort_values(by='review_scores_rating', ascending=False)
other_rooms = df_filtered[df_filtered['room_type'] != 'Shared room'].sort_values(by='price')

# Combinar dando prioridad a compartidas con mejor puntuación
result = pd.concat([shared_rooms, other_rooms]).head(10)

# Mostrar resultados
print(result[['name', 'price', 'room_type', 'review_scores_rating']])


### 3. Realizar Agrupamientos de Datos


- Según su criterio, aplicar 2 tipos de agrupamientos a los datos brindados. 
- **Opcional**, averiguar como realizar el gráfico de los 2 agrupamientos de los datos realizados. Se podría emplear librerias como mathplot y searbon ... ayudarse de chatgtp, etc


In [ ]:
data = pd.DataFrame({
    'genero': ['M', 'F', 'M', 'F'],
    'edad': [25, 25, 30, 30],
    'ingresos': [2000, 2200, 2500, 2700]
})
sns.barplot(data=data, x='edad', y='ingresos', hue='genero')
